In [ ]:
#1. Connect to SFTP and scan the receive folder for files.
#2. Upsert unseen files into `ingestion_manifest` with status=NEW.
#3. Download and stage NEW + unqueued files locally and upsert them into `pending_ingest_queue`.

#Recent refactor:
#- SFTP helpers moved to `helper.py` (`connect_sftp`, `list_receive_files`, `download_sftp_atomic`).
#- `list_receive_files` now takes `source_system` explicitly (no hidden notebook globals).

#Constraints:
# - SFTP connection required
# - NO API calls
# - Stages files locally (TMP_DIR) + writes to Delta tables only

#Inputs:
#- SFTP folder: `./receive`

#Outputs:
#- `staging_sst_01.default.ingestion_manifest`
#- `staging_sst_01.default.pending_ingest_queue`
#- Staged files written to: `/tmp/pdp_sftp_stage`


In [0]:
%pip install paramiko python-box pyyaml

In [0]:
%restart_python

In [0]:
import os
import yaml
import paramiko
from box import Box
from datetime import datetime, timezone
from databricks.connect import DatabricksSession

from pyspark.sql import functions as F
from pyspark.sql import types as T

from helper import CustomLogger, connect_sftp, list_receive_files, download_sftp_atomic

try:
    dbutils  # noqa: F821
except NameError:
    from unittest.mock import MagicMock

    dbutils = MagicMock()
spark = DatabricksSession.builder.getOrCreate()


In [0]:
logger = CustomLogger()

# Config + Secrets (kept consistent with existing pipeline)
with open("gcp_config.yaml", "rb") as f:
    cfg = Box(yaml.safe_load(f))

asset_scope = cfg.institution.secure_assets["scope"]

host = dbutils.secrets.get(scope=asset_scope, key=cfg.pdp.secret["keys"]["host"])
user = dbutils.secrets.get(scope=asset_scope, key=cfg.pdp.secret["keys"]["user"])
password = dbutils.secrets.get(
    scope=asset_scope, key=cfg.pdp.secret["keys"]["password"]
)

remote_folder = "./receive"
source_system = "NSC"

CATALOG = "staging_sst_01"
DEFAULT_SCHEMA = "default"
MANIFEST_TABLE = f"{CATALOG}.{DEFAULT_SCHEMA}.ingestion_manifest"
QUEUE_TABLE = f"{CATALOG}.{DEFAULT_SCHEMA}.pending_ingest_queue"

TMP_DIR = "/tmp/pdp_sftp_stage"

logger.info("SFTP secured assets loaded successfully.")


In [0]:
# moved to helper.py: connect_sftp


In [0]:
def ensure_tables():
    """
    Create required delta tables if missing.
    - ingestion_manifest: includes file_fingerprint for idempotency
    - pending_ingest_queue: holds local tmp path so downstream doesn't connect to SFTP again
    """
    spark.sql(
        f"""
        CREATE TABLE IF NOT EXISTS {MANIFEST_TABLE} (
          file_fingerprint STRING,
          source_system STRING,
          sftp_path STRING,
          file_name STRING,
          file_size BIGINT,
          file_modified_time TIMESTAMP,
          ingested_at TIMESTAMP,
          processed_at TIMESTAMP,
          status STRING,
          error_message STRING
        )
        USING DELTA
        """
    )

    spark.sql(
        f"""
        CREATE TABLE IF NOT EXISTS {QUEUE_TABLE} (
          file_fingerprint STRING,
          source_system STRING,
          sftp_path STRING,
          file_name STRING,
          file_size BIGINT,
          file_modified_time TIMESTAMP,
          local_tmp_path STRING,
          queued_at TIMESTAMP
        )
        USING DELTA
        """
    )


In [0]:
# moved to helper.py: list_receive_files


In [0]:
def build_listing_df(file_rows):
    schema = T.StructType(
        [
            T.StructField("source_system", T.StringType(), False),
            T.StructField("sftp_path", T.StringType(), False),
            T.StructField("file_name", T.StringType(), False),
            T.StructField("file_size", T.LongType(), True),
            T.StructField("file_modified_time", T.TimestampType(), True),
        ]
    )

    df = spark.createDataFrame(file_rows, schema=schema)

    # Stable fingerprint from metadata (file version identity)
    # Note: cast mtime to string in a consistent format to avoid subtle timestamp formatting diffs.
    df = df.withColumn(
        "file_fingerprint",
        F.sha2(
            F.concat_ws(
                "||",
                F.col("source_system"),
                F.col("sftp_path"),
                F.col("file_name"),
                F.coalesce(F.col("file_size").cast("string"), F.lit("")),
                F.coalesce(
                    F.date_format(
                        F.col("file_modified_time"), "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"
                    ),
                    F.lit(""),
                ),
            ),
            256,
        ),
    )

    return df


In [0]:
def upsert_new_to_manifest(df_listing):
    """
    Insert NEW rows for unseen fingerprints only.
    """
    df_manifest_insert = (
        df_listing.select(
            "file_fingerprint",
            "source_system",
            "sftp_path",
            "file_name",
            "file_size",
            "file_modified_time",
        )
        .withColumn("ingested_at", F.lit(None).cast("timestamp"))
        .withColumn("processed_at", F.lit(None).cast("timestamp"))
        .withColumn("status", F.lit("NEW"))
        .withColumn("error_message", F.lit(None).cast("string"))
    )

    df_manifest_insert.createOrReplaceTempView("incoming_manifest_rows")

    spark.sql(
        f"""
        MERGE INTO {MANIFEST_TABLE} AS t
        USING incoming_manifest_rows AS s
        ON t.file_fingerprint = s.file_fingerprint
        WHEN NOT MATCHED THEN INSERT *
        """
    )


In [0]:
def get_files_to_queue(df_listing):
    """
    Return files that should be queued for downstream processing.

    Criteria:
      - present in current SFTP listing (df_listing)
      - exist in manifest with status = 'NEW'
      - NOT already present in pending_ingest_queue
    """
    manifest_new = (
        spark.table(MANIFEST_TABLE)
        .select("file_fingerprint", "status")
        .where(F.col("status") == F.lit("NEW"))
        .select("file_fingerprint")
    )

    already_queued = spark.table(QUEUE_TABLE).select("file_fingerprint").distinct()

    # Only queue files that are:
    #   in current listing AND in manifest NEW AND not in queue
    to_queue = df_listing.join(manifest_new, on="file_fingerprint", how="inner").join(
        already_queued, on="file_fingerprint", how="left_anti"
    )
    return to_queue


In [0]:
# moved to helper.py: _hash_file, _remote_hash, download_sftp_atomic


In [0]:
def download_new_files_and_queue(sftp: paramiko.SFTPClient, df_new):
    """
    Download each new file to /tmp and upsert into pending_ingest_queue.
    """
    os.makedirs(TMP_DIR, exist_ok=True)

    # Collect is OK if you expect modest number of files. If you expect thousands, we can paginate and stream.
    rows = df_new.select(
        "file_fingerprint",
        "source_system",
        "sftp_path",
        "file_name",
        "file_size",
        "file_modified_time",
    ).collect()

    queued = []
    for r in rows:
        fp = r["file_fingerprint"]
        sftp_path = r["sftp_path"]
        file_name = r["file_name"]

        remote_path = f"{sftp_path.rstrip('/')}/{file_name}"
        local_path = os.path.abspath(os.path.join(TMP_DIR, f"{fp}__{file_name}"))

        # If local already exists (e.g., rerun), skip re-download
        if not os.path.exists(local_path):
            print(f"Downloading new file from SFTP: {remote_path} -> {local_path}")
            logger.info(
                f"Downloading new file from SFTP: {remote_path} -> {local_path}"
            )
            # sftp.get(remote_path, local_path)
            download_sftp_atomic(sftp, remote_path, local_path, chunk=150)
        else:
            print(f"Skipping download, file already exists: {local_path}")
            logger.info(f"Local file already staged, skipping download: {local_path}")

        queued.append(
            {
                "file_fingerprint": fp,
                "source_system": r["source_system"],
                "sftp_path": sftp_path,
                "file_name": file_name,
                "file_size": r["file_size"],
                "file_modified_time": r["file_modified_time"],
                "local_tmp_path": local_path,
                "queued_at": datetime.now(timezone.utc),
            }
        )

    if not queued:
        return 0

    qschema = T.StructType(
        [
            T.StructField("file_fingerprint", T.StringType(), False),
            T.StructField("source_system", T.StringType(), False),
            T.StructField("sftp_path", T.StringType(), False),
            T.StructField("file_name", T.StringType(), False),
            T.StructField("file_size", T.LongType(), True),
            T.StructField("file_modified_time", T.TimestampType(), True),
            T.StructField("local_tmp_path", T.StringType(), False),
            T.StructField("queued_at", T.TimestampType(), False),
        ]
    )

    df_queue = spark.createDataFrame(queued, schema=qschema)
    df_queue.createOrReplaceTempView("incoming_queue_rows")

    # Upsert into queue (idempotent by fingerprint)

    spark.sql(
        f"""
        MERGE INTO {QUEUE_TABLE} AS t
        USING incoming_queue_rows AS s
        ON t.file_fingerprint = s.file_fingerprint
        WHEN MATCHED THEN UPDATE SET
        t.local_tmp_path = s.local_tmp_path,
        t.queued_at = s.queued_at
        WHEN NOT MATCHED THEN INSERT *
        """
    )

    return len(queued)


In [0]:
transport = None
sftp = None

try:
    ensure_tables()

    transport, sftp = connect_sftp(host, user, password)
    logger.info(f"Connected to SFTP host={host} and scanning folder={remote_folder}")

    file_rows = list_receive_files(sftp, remote_folder, source_system)
    if not file_rows:
        logger.info(f"No files found in SFTP folder: {remote_folder}. Exiting (no-op).")
        dbutils.notebook.exit("NO_FILES")

    df_listing = build_listing_df(file_rows)

    # 1) Ensure everything on SFTP is at least represented in manifest as NEW
    upsert_new_to_manifest(df_listing)

    # 2) Queue anything that is still NEW and not already queued
    df_to_queue = get_files_to_queue(df_listing)

    to_queue_count = df_to_queue.count()
    if to_queue_count == 0:
        logger.info(
            "No files to queue: either nothing is NEW, or NEW files are already queued. Exiting (no-op)."
        )
        dbutils.notebook.exit("QUEUED_FILES=0")

    logger.info(
        f"Queuing {to_queue_count} NEW-unqueued file(s) to {QUEUE_TABLE} and staging locally."
    )
    queued_count = download_new_files_and_queue(sftp, df_to_queue)

    logger.info(
        f"Queued {queued_count} file(s) for downstream processing in {QUEUE_TABLE}."
    )
    dbutils.notebook.exit(f"QUEUED_FILES={queued_count}")

finally:
    try:
        if sftp is not None:
            sftp.close()
    except Exception:
        pass
    try:
        if transport is not None:
            transport.close()
    except Exception:
        pass
